In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
import itertools
import random as rn
import tensorflow as tf
%matplotlib inline

C:\Users\sjcrum\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
### Reproducibility ###

#import random as rn
#import tensorflow as tf
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(29)  # For numpy numbers
rn.seed(29)   # For Python
tf.set_random_seed(29)    #For Tensorflow


In [3]:
#Force tensorflow to use a single thread
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

#Keras code goes after this

In [3]:
os.chdir("C:/Users/sjcrum/Documents\GitHub\Plant-Image-Recognition\MLTutorial")
train_path = "cats_and_dogs/train"
test_path = "cats_and_dogs/test"
valid_path = "cats_and_dogs/valid"

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:/Users/sjcrum/Documents/Data_Science_Capstone/MLTutorial'

In [ ]:
#Generates batches of tensor image data that images must be in
#F-f-d takes path and puts in batches of normalized data, one-hot encodes classes (class_mode argument)
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size = (224,224), batch_size = 1000)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size = (224,224), batch_size = 5)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size = (224,224), batch_size = 250)

In [ ]:
def plots(ims, figsize = (12,6), rows = 1, interp = False, title = None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize = figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if title is not None:
            sp.set_title(title[i], fontsize = 10)
        plt.imshow(ims[i], interpolation = None if interp else 'none')
    
    

In [ ]:
ims, labels = next(train_batches)

In [ ]:
plots(ims, title = labels)

In [ ]:
#### Build and Train CNN ####

model = Sequential([])

model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (224,224,3), padding='same'))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(2, activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
#Compile model
#Arguments = optimizer(learning rate), loss, metrics
model.compile(Adam(lr = 0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
#Steps per epoch = training size / batch size
model.fit_generator(train_batches, steps_per_epoch=44, validation_data = valid_batches, validation_steps=31, epochs = 5, verbose = 2)

In [ ]:
#### Predicting ####
test_ims, test_labels = next(test_batches)
plots(test_ims, title = test_labels)

In [ ]:
# First column, class is 1d 0 or 1
test_labels = test_labels[:, 0]
test_labels

In [ ]:
predictions = model.predict_generator(test_batches, steps = 1, verbose = 0)
predictions

In [4]:
cm = confusion_matrix(test_labels, predictions[:,0])

NameError: name 'test_labels' is not defined

In [5]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [6]:
cm_plot_labels = ["No Side Effects", "Had Side Effects"]
plot_confusion_matrix(cm, cm_plot_labels, title = "Confusion Matrix")

NameError: name 'cm' is not defined

In [7]:
model.save("Med_VGG16_model.h5")

NameError: name 'model' is not defined